# Hydrology skill assessment

In [ ]:
import pandas as pd
import mikeio
import modelskill as ms

In [ ]:
idx = 6458010

ds = mikeio.read(f"examples/hydrology/data/{idx}.dfs0")
ds


In [ ]:
cmp = ms.from_matched("examples/hydrology/data/6458010.dfs0", obs_item=0, mod_items=[1,2], name="TCEW")
cmp

In [ ]:
cmp.plot.timeseries(backend="plotly")

In [ ]:
cmp.plot.scatter();

In [ ]:
cmp.plot.kde();

In [ ]:
cmp.plot.qq();

In [ ]:
cmp.skill()

In [ ]:
cmp.skill(metrics=["kge","cc"])

## Many locations

In [ ]:
stations = pd.read_csv("examples/hydrology/stations.csv").sort_values("weight", ascending=False)
stations.head()

In [ ]:
cmps = []
for index, row in stations.iterrows():
    cmp = ms.from_matched(f"examples/hydrology/data/{row.id}.dfs0",x=row.lon,y=row.lat,obs_item=0, mod_items=[1,2], name=row['name'])
    cmps.append(cmp)

In [ ]:
cc = ms.ComparerCollection(cmps)
cc

In [ ]:
weights = dict(zip(stations.name, stations.weight))
weights

In [ ]:
cc.score(metric="kge", weights='equal')

In [ ]:
cc.score(metric="kge", weights=weights)

In [ ]:
gdf = (cc.sel(model="sim1")
 .skill(metrics="kge")
 .to_geodataframe()
)
gdf.head()

In [ ]:
cc.skill(metrics="kge")["kge"].plot.barh()

In [ ]:
gdf.explore("kge", marker_kwds={"radius":10}, tiles="CartoDB positron", cmap="RdYlGn")

In [ ]:
sdf = cc.skill(metrics="kge").to_dataframe().sort_values("kge", ascending=True).reset_index()

import plotly.express as px

px.scatter(sdf, x="kge", y="observation",color='model', width=600)

